In [57]:
import openpyxl
import pandas as pd
from openpyxl.worksheet.worksheet import Worksheet

file_name = "synthtic_data_economy.xlsx"
workbook = openpyxl.load_workbook(file_name)


def extract_dataframes(ws: Worksheet, sheet_index: int, file_name: str):
    skip_index = 0
    for row_values in ws.iter_rows(values_only=True):
        if "Projekt" in row_values:
            break
        skip_index += 1

    # extract dataframe, and drop their last row, remove invalid rows, then set Projekt column as index
    return (
        pd.read_excel(file_name, sheet_name=sheet_index, skiprows=skip_index)[:-1]
        .dropna(subset=["Projekt"])
        .drop_duplicates(subset=["Projekt"])
        .set_index("Projekt")
    )


df_list = [
    extract_dataframes(ws, idx, file_name) for idx, ws in enumerate(workbook.worksheets)
]

In [58]:
df_list[0]

,PROJL,AlphaRate,contract start,contract end,NodeCount,TempValue,SignalType,BatchID,CoreMass,DriftUnit,...,LoadFactor,SpinRate,ByteSize,EchoTone,GripLevel,PathCode,LineFreq,SnapMode,PulseRate,ZoneTag
Projekt,,,,,,,,,,,,,,,,,,,,,
482193 Lantern,TA417 Tohid Ardeshiri,0.870000,2021-01-27,2023-06-12,12.000000,23.500000,Analog,B-102,4.200000,0.003000,...,1.500000,9800.000000,512.000000,Soft,Medium,PC-88,60Hz,Auto,1.030000,ZN-A
903284 Summit,A9X4B2 Marcus Ellington,0.450000,2020-02-01,2023-10-07,8.000000,19.800000,Digital,B-103,3.900000,0.005000,...,1.200000,8700.000000,256.000000,Loud,High,PC-89,50Hz,Manual,0.980000,ZN-B
174820 Velvet,ZQ7L8M Fiona Caldwell,0.920000,2021-10-29,2020-11-20,15.000000,21.100000,Analog,B-104,4.500000,0.002000,...,1.700000,9900.000000,1024.000000,Mute,Low,PC-90,55Hz,Auto,1.100000,ZN-C
621937 Orbit,B3T9KX Jerome Whitaker,0.245579,2021-08-18,2023-04-13,0.351951,0.709527,0.198383,0.005782,0.971812,0.871315,...,0.726278,0.993495,0.749258,0.055047,0.557317,0.466651,0.709584,0.558006,0.365685,0.121824
398120 Timber,L8N2YD Clara Voss,0.210654,2023-03-04,2022-08-08,0.577274,0.595901,0.467559,0.004627,0.980756,0.002359,...,0.268408,0.790518,0.640808,0.426545,0.430334,0.641086,0.815614,0.343713,0.039739,0.349686
740293 Quartz,X5P0QJ Damien Rhodes,0.632242,2024-02-05,2024-08-14,0.549121,0.506249,0.930073,0.121448,0.372655,0.429375,...,0.007084,0.806209,0.708166,0.079107,0.348311,0.00273,0.933292,0.305972,0.167817,0.081836
219384 Meadow,M7C1VE Natalie Brooks,0.944451,2023-11-21,2022-01-02,0.665218,0.038540,0.969371,0.189362,0.383145,0.877051,...,0.062400,0.175613,0.508973,0.469693,0.941327,0.867167,0.718422,0.128483,0.577093,0.783305
581029 Pixel,R2D8WZ Oliver Trent,0.876822,2022-01-03,2023-11-01,0.649144,0.459746,0.662688,0.000832,0.252843,0.193595,...,0.518250,0.324602,0.215537,0.957988,0.510941,0.178971,0.55466,0.024541,0.479415,0.563338
347812 Canyon,K9J3UF Sabrina Holt,0.022086,2023-11-02,2022-05-21,0.526723,0.854130,0.698709,0.335949,0.009031,0.503672,...,0.434180,0.618540,0.038056,0.286292,0.047852,0.547868,0.917461,0.200175,0.145996,0.895287


In [66]:
def join_two_dataframe(
    df_1: pd.DataFrame, df_2: pd.DataFrame, diff_warning: dict | None = None
):
    merged = pd.merge(df_1, df_2, left_index=True, right_index=True, how='outer')
    if diff_warning is None:
        diff_warning = {}

    for col in df_1.columns.intersection(df_2.columns):
        filtered = merged[
            merged[f"{col}_x"].notna()
            & merged[f"{col}_y"].notna()
            & (merged[f"{col}_x"] != merged[f"{col}_y"])
        ]
        filtered = filtered[[f"{col}_x", f"{col}_y"]]
        for idx, data in filtered.iterrows():
            if idx not in diff_warning:
                diff_warning[idx] = {}
            if col not in diff_warning[idx]:
                diff_warning[idx][col] = set()

            diff_warning[idx][col].add(data[f"{col}_x"])
            diff_warning[idx][col].add(data[f"{col}_y"])

        merged[col] = merged[f"{col}_x"].combine_first(merged[f"{col}_y"])
        merged = merged.drop(columns=[f"{col}_x", f"{col}_y"])
    return diff_warning, merged


warning_dict = {}
warning_dict, merge = join_two_dataframe(df_list[0], df_list[1], warning_dict)
warning_dict, merge = join_two_dataframe(merge, df_list[2], warning_dict)

In [67]:
from pprint import pprint 
pprint(warning_dict)

{'129384 Echo  ': {'AlphaRate': {np.float64(0.047592122),
                                 np.float64(0.760181156),
                                 np.float64(0.95708641)},
                   'contract end': {Timestamp('2020-05-27 00:00:00'),
                                    Timestamp('2022-11-06 00:00:00'),
                                    Timestamp('2024-11-19 00:00:00')},
                   'contract start': {Timestamp('2022-02-20 00:00:00'),
                                      Timestamp('2023-09-25 00:00:00'),
                                      Timestamp('2023-11-06 00:00:00')}},
 '129384 Mirage  ': {'AlphaRate': {np.float64(0.184216957),
                                   np.float64(0.446356566),
                                   np.float64(0.838973412)},
                     'contract end': {Timestamp('2021-04-27 00:00:00'),
                                      Timestamp('2022-04-11 00:00:00'),
                                      Timestamp('2023-11-20 00:00:00')},

In [69]:
merge

,NodeCount,TempValue,SignalType,BatchID,CoreMass,DriftUnit,PhaseMark,LoadFactor,SpinRate,ByteSize,...,Phone,Status,Score,Rank,Category,Group,PROJL,AlphaRate,contract start,contract end
Projekt,,,,,,,,,,,,,,,,,,,,,
129384 Echo,0.593475,0.026066,0.587294,0.13088,0.968051,0.652980,0.721156,0.527590,0.821617,0.470008,...,0.622916,0.913294,0.796208,0.045070,0.073905,0.202641,Y1V9QX Giselle Monroe,0.047592,2022-02-20,2022-11-06
129384 Mirage,0.753423,0.041474,0.180145,0.727991,0.672977,0.400555,0.672916,0.056064,0.524642,0.719285,...,0.563563,0.313434,0.108427,0.454951,0.698175,0.953437,B4T1XJ Leo Sanderson,0.446357,2022-06-13,2021-04-27
174820 Timber,0.127023,0.627356,0.519903,0.968268,0.449957,0.218221,0.641078,0.814468,0.434505,0.718831,...,0.340259,0.927662,0.230093,0.278575,0.851496,0.37419,K2L9VX Dahlia West,0.698010,2020-08-08,2024-04-16
174820 Velvet,15.000000,21.100000,Analog,B-104,4.500000,0.002000,PM-5,1.700000,9900.000000,1024.000000,...,PM-5,1.700000,9900.000000,1024.000000,Mute,Low,ZQ7L8M Fiona Caldwell,0.920000,2021-10-29,2020-11-20
193840 Grove,0.627440,0.074619,0.010666,0.544729,0.509722,0.673407,0.635444,0.126386,0.450863,0.790412,...,0.486784,0.207643,0.634016,0.875867,0.219953,0.705255,G3T0KU Lila Emerson,0.720312,2024-02-25,2022-05-15
219384 Meadow,0.665218,0.038540,0.969371,0.189362,0.383145,0.877051,0.128821,0.062400,0.175613,0.508973,...,0.060894,0.763893,0.353532,0.970628,0.16177,0.425096,M7C1VE Natalie Brooks,0.944451,2023-11-21,2022-01-02
239481 Drift,0.275658,0.840530,0.959102,0.55095,0.084488,0.734854,0.537088,0.354807,0.261000,0.591925,...,0.041628,0.806101,0.533330,0.033466,0.66874,0.324343,S7N3QY Felix Durant,0.309166,2022-04-07,2022-03-02
283910 Glimmer,0.297243,0.168290,0.38084,0.334989,0.819494,0.527054,0.882539,0.670397,0.534469,0.569160,...,0.832095,0.133425,0.566546,0.899809,0.438685,0.955405,J3Y9PA Helena Cruz,0.391958,2024-10-19,2022-07-05
294183 Ripple,0.331534,0.820769,0.693332,0.07064,0.607100,0.513860,0.333513,0.280771,0.957074,0.377154,...,0.506472,0.256127,0.522289,0.748521,0.677663,0.63133,Q9M7ZL Tessa Vaughn,0.162803,2020-09-25,2023-10-02
